In [ ]:
import torch
import numpy as np
import scipy
#packages
from matplotlib import legend
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import os

import numpy as np
#from torchdyn.numerics import odeint
from torchdiffeq import odeint
from icecream import ic
from tqdm import tqdm
import scipy
from itertools import chain
import cProfile
import pstats
from get_data import  get_data

In [122]:
a = [1,2,3,4]
b = [5,6,7,24]
for i, x in enumerate(a):
    print(x/b[i]-1)
c = [x/y-1 for x,y in zip(a,b)]
c

-0.8
-0.6666666666666667
-0.5714285714285714
-0.8333333333333334


[-0.8, -0.6666666666666667, -0.5714285714285714, -0.8333333333333334]

In [ ]:
t3 = torch.tensor(np.linspace(0,20,100), requires_grad=True)
#t2 = torch.tensor(np.linspace(0,20,100), requires_grad=True, dtype=int)

y = torch.sin(t3)
s = torch.sum(y)
s.backward()
#plt.plot(y)
#ic(y, yp)
plt.plot(y.detach().numpy())
plt.plot(t3.grad.detach().numpy(), label="grad")
#plt.plot(ytest, label="test")
# plt.legend()
plt.show()

In [ ]:
t2 = torch.tensor(np.linspace(0,30,200))
#t2 = torch.tensor(np.linspace(0,20,100), requires_grad=True, dtype=int)

y = torch.sin(4*t2) + torch.cos(t2)**2

yd = torch.tensor([(y[t+1]-y[t])/(30/200) for t in range(200-1)])
ytest=4*torch.cos(4*t2) - 2*torch.cos(t2)*torch.sin(t2) 

yd = torch.gradient(input =  y, spacing=(t2,))

plt.plot(y.detach().numpy())
plt.plot(yd[0].detach().numpy())
plt.plot(ytest.detach().numpy(),linestyle="dashed")
plt.show()

In [ ]:
t = np.linspace(0,90,600)

a = np.random.uniform(0.1, 0.3, 1)
b = np.random.uniform(1, 5, 1)
c = np.random.uniform(0, 5, 1)
sign = 1 if np.random.random() < 0.5 else -1
ic(sign)
y = sign * a * np.tanh((t - 10)/5 )
# y[200:300] = 0.1 * np.cos(2 *t[200:300])**2
# y[300:500] = sign * a/100 * np.tan((t[300:500] )/10 )
# y[500:700] = sign * a/100 * (t[500:700]/10)**2

y = 0 * np.cos(t)

for t in range(1,6):
    y[100*t:100*t+100] =  np.ones(100)*np.random.uniform(-0.2, 0.2, 1)[0]

plt.plot(y)

In [ ]:
greek_letterz=[chr(code) for code in range(945,970)] #7 theta 24 omega

print(greek_letterz)
for i,x in enumerate(greek_letterz):
    print(i,x)

In [ ]:

    input_data, test_data, time, initial_values, input_data_w_time = get_data(x0 = np.pi/4, y0 = 0.1, use_fixed_init = False, t0=start_time, t1=stop_time, 
                                                                                   time_steps=timesteps, num_of_inits=num_of_inits,
                                                                                     normalize=False, add_noise=False, u_option="tanh",  set_seed=False)

In [ ]:

def input(t, u_option):
  
  if u_option=="noise":
   return np.random.normal(0, 0.2, len(t))
  
  if u_option=="sin":
   return 0.1 * np.sin(t)
  
  if u_option=="cos":
   return np.cos(t)
  

  else: 
   return 0
  

def func(y,t, u):
  f = 1
  #0.5
  
  return np.array([y[1], 1/f*(-np.sin(y[0])- 1/7*y[1] + u )])


In [ ]:
t =np.linspace(0,30,20)
steps = np.random.uniform(-0.02, 0.02, len(t))
steps[0] = np.random.uniform(-0.2, 0.2, 1)
walk = np.cumsum(steps)
walk


#0.1 * np.sin(t)

In [ ]:
a = func(np.array([1, 0.1]), t=5 ,args="noise")
b = func(a, t=5 ,args="noise")
a,b

In [ ]:

n=400

t = np.linspace(0, 60, n)

u_option = "sin"
#
for j in range(1):
    y0_list_w = np.random.uniform(-np.pi/3, np.pi/3, size=(1, 1))
    y0_list_o = np.random.uniform(-0.1, 0.1, size=(1, 1))
    y0_list = np.concatenate([y0_list_w, y0_list_o], axis=1)

    z0 = y0_list[0]
    u = input(t, u_option)

    #u = np.sin(3*t)/2
    #u = (t**2 - 2*t)*0.0001

    # 
    out = np.zeros((len(t), 2))
    out[0] = z0

    for i in range(1, n):
        t_span = [t[i-1], t[i]]
        z = scipy.integrate.odeint(func, out[i-1], t_span, args = (u[i],))
        out[i] = z[1]

    plt.plot(t,out[:,0])
plt.grid()
plt.show()

In [ ]:
input_data, test_data, time, initial_values = generate_data(x0 = np.pi/4, y0 = 0.1, use_fixed_init = False, t0=0, t1=60, 
                                           time_steps=200, num_of_inits=10, normalize=True, add_noise=False, u_option="noise")

ic(input_data.size())
ic(test_data.size())

In [ ]:
for x in input_data:
    plt.plot(time, x[:,2])

plt.show()

In [ ]:
train_size = int(0.8 * len(input_data))
test_size = len(input_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(input_data, [train_size, test_size])

ic(train_dataset)
for x in train_dataset:
    ic(x)

In [ ]:

def get_data(x0 = np.pi/4, y0 = 0.1, use_fixed_init = False, t0=0, t1=30, time_steps=1000, num_of_inits=1, normalize=True, add_noise=False, u_option="noise"):

    np.random.seed(seed=42)

    
    y0 = [x0, y0]
    
    t = np.linspace(t0, t1,num=time_steps)
    y0_list_w = np.random.uniform(-np.pi/1, np.pi/1, size=(num_of_inits, 1)) 
    y0_list_o = np.random.uniform(-0.2, 0.2, size=(num_of_inits, 1)) 
    y0_list = np.concatenate([y0_list_w, y0_list_o], axis=1)
    
    if use_fixed_init:
      y0_list = [y0]
      
    trajectory_list = []
    input_data_list = []
    input_data_w_time_list = []

    for i, y0 in enumerate(y0_list):

      np.random.seed(seed=i)
      
      out = np.zeros((len(t), 2))
     
      out[0] = y0_list[i]
      
      u = input(t, u_option)
     
      for i in range(1, len(t)):

        t_span = [t[i-1], t[i]]
        z = scipy.integrate.odeint(func, out[i-1], t_span, args = (u[i],))

        out[i] = z[1]

      out = torch.tensor(out)
      u = torch.tensor(u).view(1,len(t))
      u = u.transpose(0,1)
      time = torch.tensor(t)
      time = time.view(1,len(t))
      time = time.transpose(0,1)

      trajectory_list.append(out)
      input_data_list.append(torch.cat((u,out), dim=-1))
      input_data_w_time_list.append(torch.cat((time,u,out), dim=-1))

    test_data = torch.stack(trajectory_list)
    input_data = torch.stack(input_data_list)

    input_data_w_time = torch.stack(input_data_w_time_list)

    if normalize:
      
     test_data = torch.nn.functional.normalize(test_data) 
     input_data = torch.nn.functional.normalize(input_data) 
     input_data_w_time = torch.nn.functional.normalize(input_data_w_time) 
    #  test_data = test_data/torch.max(abs(test_data))
    #  input_data = input_data/torch.max(abs(input_data))
    #  input_data_w_time = input_data_w_time/torch.max(abs(input_data_w_time))

     
    return input_data, test_data, time, torch.tensor(y0_list), input_data_w_time



In [ ]:
window_size = 1
start_time = 0
stop_time = 10
timesteps = 15
num_of_inits = 3
option_odestep = True
losses=[]

input_data, test_data, time, initial_values, input_data_w_time = get_data(x0 = np.pi/4, y0 = 0.1, use_fixed_init = False, t0=start_time, t1=stop_time, 
                                                                                time_steps=timesteps, num_of_inits=num_of_inits, normalize=False, add_noise=False, u_option="noise")


train_size = int(0.8 * len(input_data))
test_size = len(input_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(input_data, [train_size, test_size])

In [ ]:
train_dataset[:][:,0:8, :]

In [ ]:
torch.manual_seed(3)
a = torch.rand(20, 100, 2)*15
b = torch.rand(20, 100, 2)*6
a=a-b
ic(a.size(dim=1))
#a = a/torch.max(abs(a))

a = torch.nn.functional.normalize(a)
#ic(a)
for i in range(20):
    print(np.linalg.norm(a[i,:,0].numpy()), torch.mean(a[i,:,0]))
